In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib.ticker import FuncFormatter
from IPython.display import HTML
# =========================================================
# CONFIGURATION
# =========================================================
FPS = 30                       # Playback speed
ANIMATION_DURATION = 15        # Seconds (excluding pause)
PAUSE_DURATION = 5             # Pause at final frame (seconds)

# =========================================================
# TIME AXIS
# =========================================================
years = np.array([2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025])
x_shifted = years - 0.5        # Center values between gridlines

# =========================================================
# DATA
# =========================================================
series_a = np.array([8, 15, 28, 34, 41, 48, 55, 62])   # steady long-term growth
series_b = np.array([35, 33, 29, 26, 24, 30, 38, 42]) # dip + recovery
series_c = np.array([18, 22, 19, 27, 23, 35, 49, 45]) # volatility
series_d = np.array([42, 45, 39, 36, 40, 44, 47, 50]) # stable leader

# =========================================================
# FRAME CALCULATIONS
# =========================================================
total_frames = ANIMATION_DURATION * FPS
frames_per_step = total_frames // (len(years) - 1)
pause_frames = int(PAUSE_DURATION * FPS)
total_frames_with_pause = total_frames + pause_frames

# =========================================================
# FIGURE & AXES SETUP
# =========================================================
fig, ax = plt.subplots(figsize=(13, 5))
ax.set_position([0.05, 0.1, 0.93, 0.83])

ax.set_xlim(years[0] - 1, years[-1])
ax.set_ylim(0, 100)
ax.set_yticks(np.arange(0, 101, 10))
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{int(y)}%"))

ax.set_xticks(years)
ax.set_xticks(x_shifted, minor=True)
ax.set_xticklabels(years, minor=True)

ax.tick_params(axis="x", which="major", labelbottom=False)
ax.tick_params(axis="x", which="minor", pad=10)
ax.tick_params(axis="y", pad=10)

ax.grid(True, axis="x", color="#555557", linewidth=1)
ax.grid(True, axis="y", alpha=0.3)

fig.patch.set_facecolor("#eaeaef")
ax.set_facecolor("#eaeaef")

for spine in ax.spines.values():
    spine.set_color((0.25, 0.25, 0.25, 0.3))
    spine.set_linewidth(1)

# =========================================================
# LINE STYLES
# =========================================================
line_a, = ax.plot([], [], color="#017784", linewidth=3, marker="o",
                  markersize=17, markerfacecolor="white", markeredgewidth=4,
                  label="Series A")

line_b, = ax.plot([], [], color="#538B53", linewidth=3, marker="o",
                  markersize=17, markerfacecolor="white", markeredgewidth=4,
                  label="Series B")

line_c, = ax.plot([], [], color="#E8AE96", linewidth=2, linestyle="--",
                  marker="o", markersize=10, markerfacecolor="white",
                  markeredgewidth=2, alpha=0.9, label="Series C")

line_d, = ax.plot([], [], color="#8F160D", linewidth=2, linestyle="--",
                  marker="o", markersize=10, markerfacecolor="white",
                  markeredgewidth=2, alpha=0.9, label="Series D")

# Moving markers
pt_a = ax.plot([], [], marker="o", markersize=17, color="#017784",
               markerfacecolor="white", markeredgewidth=4)[0]
pt_b = ax.plot([], [], marker="o", markersize=17, color="#538B53",
               markerfacecolor="white", markeredgewidth=4)[0]
pt_c = ax.plot([], [], marker="o", markersize=10, color="#E8AE96",
               markerfacecolor="white", markeredgewidth=2, alpha=0.9)[0]
pt_d = ax.plot([], [], marker="o", markersize=10, color="#8F160D",
               markerfacecolor="white", markeredgewidth=2, alpha=0.9)[0]

# =========================================================
# INITIALIZATION
# =========================================================
def init():
    for line in (line_a, line_b, line_c, line_d):
        line.set_data([], [])
    for pt in (pt_a, pt_b, pt_c, pt_d):
        pt.set_data([], [])
    return line_a, line_b, line_c, line_d, pt_a, pt_b, pt_c, pt_d

# =========================================================
# ANIMATION FUNCTION
# =========================================================
def animate(frame):
    if frame >= (len(years) - 1) * frames_per_step:
        frame = (len(years) - 1) * frames_per_step - 1

    idx = frame / frames_per_step
    lower = int(np.floor(idx))
    upper = min(lower + 1, len(years) - 1)
    frac = idx - lower

    x_now = x_shifted[lower] + frac * (x_shifted[upper] - x_shifted[lower])

    def interp(data):
        return data[lower] + frac * (data[upper] - data[lower])

    y_a, y_b, y_c, y_d = map(interp, [series_a, series_b, series_c, series_d])

    x_draw = np.append(x_shifted[:upper], x_now)

    line_a.set_data(x_draw, np.append(series_a[:upper], y_a))
    line_b.set_data(x_draw, np.append(series_b[:upper], y_b))
    line_c.set_data(x_draw, np.append(series_c[:upper], y_c))
    line_d.set_data(x_draw, np.append(series_d[:upper], y_d))

    pt_a.set_data([x_now], [y_a])
    pt_b.set_data([x_now], [y_b])
    pt_c.set_data([x_now], [y_c])
    pt_d.set_data([x_now], [y_d])

    return line_a, line_b, line_c, line_d, pt_a, pt_b, pt_c, pt_d

# =========================================================
# BUILD & DISPLAY ANIMATION
# =========================================================
animation = FuncAnimation(
    fig,
    animate,
    init_func=init,
    frames=total_frames_with_pause,
    interval=1000 / FPS,
    blit=False,
    repeat=True
)

# gif_path = "animation.gif"

# gif_writer = PillowWriter(
#     fps=FPS,
#     metadata={"artist": "Matplotlib"},
#     bitrate=1800
# )

# animation.save(gif_path, writer=gif_writer)
# print(f"✅ GIF saved: {gif_path}")

# Save MP4
# writer = FFMpegWriter(
#     fps=FPS,
#     codec="libx264",
#     bitrate=2500,
#     extra_args=["-pix_fmt", "yuv420p"]
# )

# animation.save("animation.mp4", writer=writer, dpi=300)
# print("✅ MP4 saved: animation.mp4")

plt.close(fig)
HTML(animation.to_html5_video())


✅ MP4 saved: animation.mp4


In [8]:
# from google.colab import files
# files.download('animation.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>